# Extract splice event features
Given a set of RMATS junction count files, aggregate and remove duplicate events. Write these aggregated events to file.

**Runtime**: < 5 minutes (I/O limited)

In [4]:
%load_ext autoreload
%autoreload 2
%load_ext autotime

import os
import sys
sys.path.append('..')

from splintr import splice
from splintr.util.splice import SpliceData

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from shutil import copyfile
from tqdm.autonotebook import tqdm

import multiprocessing
from multiprocessing import Pool

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime


/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/tqdm/autonotebook/__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


ImportError: cannot import name 'splice'

time: 2.5 s


In [5]:
def remove_duplicate_events(df):
    duplicated = df.iloc[:, 5:-14].duplicated()
    df = df.loc[~duplicated]
    return df

time: 1.34 ms


In [44]:
rmats_dir = '../data/encore_rmats'
output_dir = '../data/features'

# Iterate over event -> cell line -> RBP
def generate_features(event):
    cell_lines = os.listdir(f'{rmats_dir}/{event}')
    
    # cell line
    event_all_cell_dataset = []
    for cell_line in tqdm(cell_lines, position=1, desc='Cell lines', leave=False):
        jc_files = os.listdir(f'{rmats_dir}/{event}/{cell_line}')
        
        # sample
        cell_datasets = []
        for jc_file in tqdm(jc_files, position=2, desc='Samples', leave=False):
            jc_filepath = f'{rmats_dir}/{event}/{cell_line}/{jc_file}'
            sample_name = jc_file.split('.')[0] # parse sample name
            
            # Load junction counts data
            jc = pd.read_csv(jc_filepath, sep='\t')
            jc['event'] = event
            
            # Set aside non-alternatively spliced events as control
            bg_jc = jc.loc[jc['FDR'] > 0.1]
            bg_jc = bg_jc.iloc[np.random.randint(bg_jc.shape[0], size=1)]
            bg_jc['sample'] = 'bg'
            
            # Alternatively spliced events
            jc = jc.loc[jc['FDR'] < 0.1]
            jc['sample'] = sample_name
            jc = pd.concat([bg_jc, jc])
            
            cell_datasets.append(jc)
            
        # Combine all events for event type
        cell_datasets = pd.concat(cell_datasets, ignore_index=True)
        
        # Remove duplicates
        cell_datasets = remove_duplicate_events(cell_datasets)
        event_all_cell_dataset.append(cell_datasets)
        
        # Write to file
        file_prefix = f'{output_dir}/{cell_line}_{event}'
        cell_datasets.to_csv(f'{file_prefix}.txt', sep='\t', index=False)

    # Combine all events for given event type across all cell lines
    event_all_cell_dataset = pd.concat(event_all_cell_dataset, ignore_index=True)
    
    # remove duplicates
    event_all_cell_dataset = remove_duplicate_events(event_all_cell_dataset)
    event_all_cell_dataset.to_csv(f'{output_dir}/{event}.txt', sep='\t', index=False)
    
events = os.listdir(rmats_dir)
p = Pool(4)
list(tqdm(p.imap_unordered(generate_features, events), total=len(events)))

[None, None, None, None, None]

time: 4min


In [2]:
data = pd.read_csv('../data/features/RI.txt')

NameError: name 'pd' is not defined

time: 8.26 ms
